# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 ноября 2017, 23:59   
**Штраф за опоздание:** -2 балла после 23:59  4 декабря, -4 балла после 23:59 11 декабря, -6 баллов после 23:59 18 декабря

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

**Разбаловка:**   
За задание можно получить 10 баллов. Для этого нужно следующее:
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает    
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала качественнее, чем у дерева из sklearn

In [1]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext Cython

In [70]:
%%cython -a
import numpy as np
cimport numpy as np
import cython

@cython.boundscheck(False)
@cython.cdivision(True)
@cython.wraparound(False)
cpdef int uniq_count(int[:] y,
                     int num_class):
    cdef int[:] classes = np.zeros(num_class, dtype=np.int32)
    for i in range(y.shape[0]):
        classes[y[i]] = 1
        
    cdef int result = 0
    for i in range(num_class):
        result += 1
    return result

@cython.boundscheck(False)
@cython.cdivision(True)
@cython.wraparound(False)
cpdef void sort_samples(np.ndarray[float, ndim=1] x, int[:] y):
        cdef long[:] sorted_idx = x.argsort()
        cdef float[:] copy_x = x.copy()
        cdef int[:] copy_y = y.copy()
        for i in range(x.shape[0]):
            x[i] = copy_x[sorted_idx[i]]
            y[i] = copy_y[sorted_idx[i]]

In [72]:
import numpy as np

class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.n_feature = -1

        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    @staticmethod
    def __gain(function, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        left_measure = function(l_c / l_s[:, np.newaxis])
        right_measure = function(r_c / r_s[:, np.newaxis])
        return l_s * left_measure + r_s * right_measure

    @classmethod
    def __gini(cls, l_c, l_s, r_c, r_s):
        return cls.__gain((lambda probs: np.sum((probs) ** 2, axis=1)), l_c, l_s, r_c, r_s)

    @classmethod
    def __entropy(cls, l_c, l_s, r_c, r_s):
        def nlogn(n):
            return n * np.log(n)

        return cls.__gain((lambda probs: - np.sum(nlogn(probs), axis=1)), l_c, l_s, r_c, r_s)

    @classmethod
    def __misclass(cls, l_c, l_s, r_c, r_s):
        return cls.__gain((lambda probs: 1 - np.max(probs, axis=1)), l_c, l_s, r_c, r_s)

    def __get_feature_ids_sqrt(self):
        feature_ids = np.arange(self.n_feature, dtype=np.int32)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(self.n_feature))]

    def __get_feature_ids_log2(self):
        feature_ids = np.arange(self.n_feature, dtype=np.int32)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(self.n_feature))]

    def __get_feature_ids_N(self):
        return np.arange(self.n_feature, dtype=np.int32)

    @staticmethod
    def __div_samples(x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Сортируем x и y по x, получаем число классов
        sort_samples(x, y)
        class_number = uniq_count(y, self.num_class)

        # Получаем список валидных границ
        splitted_sorted_y = y[self.min_samples_split:-self.min_samples_split]
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)

        if len(r_border_ids) == 0:
            return float('+inf'), None

        # Готовим данные для подсчета границ
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(y[:self.min_samples_split], minlength=class_number)

        # Для каждой валидной границы находим число классов и размер
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(y) - l_class_count
        l_sizes = r_border_ids.reshape((l_class_count.shape[0],))
        r_sizes = y.shape[0] - l_sizes

        # Считаем gain и находим минимальный
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # Возвращаем минимальный gain и соответствующий threshold
        left_el_id = l_sizes[idx]
        return gs[idx], (x[left_el_id-1] + x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node

        if ((self.max_depth is None or depth < self.max_depth) and
            (x.shape[0] > 2 * self.min_samples_split)):

            feature_ids = self.get_feature_ids()
            min_gain = float('+inf')
            node = None

            for feature_id in feature_ids:
                gain, threshold = self.__find_threshold(x[:, feature_id], y)
                if gain < min_gain and threshold is not None:
                    min_gain = gain
                    node = [self.NON_LEAF_TYPE, feature_id, threshold]

            if node is not None:
                left_x, right_x, left_y, right_y = self.__div_samples(x, y, node[1], node[2])

                if len(left_y) > self.min_samples_split and len(right_y) > self.min_samples_split:
                    self.tree[node_id] = node
                    self.__fit_node(left_x, left_y, node_id * 2 + 1, depth + 1)
                    self.__fit_node(right_x, right_y, node_id * 2 + 2, depth + 1)

                    return

        probs = np.bincount(y, minlength=self.num_class).astype(np.float32)
        probs /= probs.sum()

        self.tree[node_id] = [self.LEAF_TYPE, np.argmax(probs), probs]

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.n_feature = x.shape[1]
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [51]:
x = df.as_matrix(columns=df.columns[1:]).astype(np.float32)
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0]).astype(np.int32)

In [73]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы

In [75]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

1.0836572647094727
7.480896234512329


## Проверка качества работы

In [7]:
gkf = KFold(n_splits=5, shuffle=True)

In [8]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

1.0
1.0
1.0
0.999958426873
0.999958425144


In [9]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.890828968155
0.893323355783
0.891826723206
0.892325600732
0.891946950484
